In [1]:
import requests,bs4,time,pyperclip
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys



In [11]:
driver_loc = "C:/Users/Lenovo/pywork/chromedriver/chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument("window-size = 1920x1080")
driver = webdriver.Chrome(driver_loc, options = options)
driver.maximize_window()
url = 'https://www.youtube.com/'
driver.get(url)
time.sleep(1.5)


inputSearch = '스마트워치'

tag_search = driver.find_element_by_name('search_query')
search_btn = driver.find_element_by_id('search-icon-legacy')

tag_search.click()
pyperclip.copy(inputSearch)
tag_search.send_keys(Keys.CONTROL, 'v')


time.sleep(0.5)

search_btn.click()
time.sleep(2)
driver.switch_to_default_content # 처음 iframe을 걷어내고 찾아야함


filter_btn = driver.find_element_by_link_text('필터')


filter_btn.click()
time.sleep(1)

# 변수로 넣어주는 경우에도 해당 요소를 찾지 못하면 계속 오류가 발생
thisYear_btn = driver.find_element_by_link_text('올해')
thisYear_btn.click()
time.sleep(1)

filter_btn.click()
time.sleep(1)

views_btn = driver.find_element_by_link_text('조회수')
views_btn.click()
time.sleep(2)

result = []
for i in range(0,5):
    column = []
    searchResult = '//*[@id="contents"]/ytd-video-renderer[{}]'.format(i+1)
    driver.find_element_by_xpath(searchResult).click()
    time.sleep(2)
    try:
        driver.find_element_by_xpath('//*[@id="content"]').click().send_keys(Keys.SPACE)
    except:
        pass
    time.sleep(3)
       
    data = driver.page_source
    bs = bs4.BeautifulSoup(data, 'html.parser')
    

    divInfoTag = bs.find(name = 'div',
                attrs = {'id': 'info-contents'})

    divMetaTag = bs.find(name = 'div',
                attrs = {'id': 'meta-contents'})

       
#     영상 제목
    a = divInfoTag.find(name = 'h1',
                   attrs = {'class': 'title style-scope ytd-video-primary-info-renderer'}).text
    
#     영상 조회수
    b = divInfoTag.find(name = 'span',
                   attrs = {'class': 'view-count style-scope yt-view-count-renderer'}).text
#     영상 날짜
    c = divInfoTag.find(name = 'div',
                       attrs = {'id': 'date'}).text.replace("•","").replace(" ","")
                   

    
#     추천수
    e = divInfoTag.find(name = 'paper-tooltip',
                       attrs = {'class': 'style-scope ytd-sentiment-bar-renderer'}).text.replace("\n","").replace(" ","")
                       
#     채널명
    d = divMetaTag.find(name = 'paper-tooltip' ,
                       attrs = {'class': 'style-scope ytd-channel-name'}).text.replace("\n","").replace(" ","")
    
    column = [d,a,b,c,e]
    result.append(column)
    
    num_page_down = 20
    while num_page_down:
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        num_page_down -= 1
        
    data = driver.page_source
    bs = bs4.BeautifulSoup(data, 'html.parser')
    #     댓글
        
    allCommentTags = bs.findAll(name = 'ytd-comment-renderer',
                               attrs = {'class': 'style-scope ytd-comment-thread-renderer'})

    rows = []
    print(len(allCommentTags))
    for j in range(0, len(allCommentTags)):
        columns1 = []
        try:
            comment = allCommentTags[j].find(name = 'div',
                                        attrs = {'id':'content',
                                                 'class':'style-scope ytd-expander'}).text.replace('\n',' ')
        except:
            comment = '1'
    
        
        colunms1 = [1,comment,3]
        rows.append(colunms1)
        
    
    result1 = pd.DataFrame(data = rows, columns=["id","댓글","공감수"])
    print(result1)
    result1.to_csv('./{}{}.csv'.format(inputSearch+"댓글",i+1), index = False )
    driver.back()

driver.close()
df_result = pd.DataFrame(data = result, columns = ['채널명','제목','조회수','날짜','추천/비추천'])
df_result.to_csv('./{}.csv'.format(inputSearch), index = False, encoding = 'ms949')




92
    id                                                 댓글  공감수
0    1                     \n\n영화관에서 영화보러갔다가 광고보고 반한 1인\n    3
1    1                          \n\n삼성은 이거 해외에도 광고내라...\n    3
2    1  \n\n이 광고 제작 관계자들 상 줘야한다. 스페인 현지에서 광고보내면 제품 조기품...    3
3    1          \n\n뭔가 시계도 패션이라는게 잘 나타나는 광고임... 계속 보게되넹\n    3
4    1                        \n\n한국에서 찍은 이국적인 광고...ㄷㄷㄷ\n    3
..  ..                                                ...  ...
87   1          \n\n이 광고보러 계속 오구 있어요 ㅎㅎ 왜이렇게 이광고가 좋지 ㅎㅎ\n    3
88   1                            \n\n제품떠나 광고 진짜 잘만들었네요\n    3
89   1  \n\n사람보는 눈 다 똑같구나 ㅎㅎ\n노래도 좋고 모델들 매력터져서\n특히 가고있...    3
90   1                     \n\n크 지려따~~~~~~~~ 느무조차너~~~♡♡\n    3
91   1                    \n\n광고 실화? 브금이랑 완전 찰떡에 모델들 굿.\n    3

[92 rows x 3 columns]
92
    id                                                 댓글  공감수
0    1  \n\n제가 바보였습니다. 갤워치도 시계줄 넣을때 안쪽 버튼을 눌러서 넣으면 쉽습니...    3
1    1                 \n\nㅋㅋㅋㅋㅋ 삼성과 애플은 충전부터 싫어해요ㅋㅋㅋ개웃\n    3
2    1  \n\n심전도 기능, 심장질환 있